##  MIND
Multi-Interest Network with Dynamic routing (MIND)

<img src="../data/img/MIND.jpeg" style="zoom:50% "/>

MIND模型的目标就是通过用户行为+用户基础属性，计算用户的K个兴趣向量 
 （当K=1时，则与YouTube DNN类似）：

- 通过Mulit-Interest Extractor Layer 获取多个向量表达用户兴趣的不同方面；
- 提出了具有动态路由的多兴趣网络（MIND），利用Dynamic Routing 以自适应地聚合用户历史行为到用户表达向量中，以处理用户的不同兴趣；
- 开发 Label-Aware Attention 标签感知注意力机制，以帮助学习具有多个向量的用户表示。


目标函数：

推荐系统召回阶段主要目标是从亿级规模的海量商品中获取与用户兴趣相关的候选集Item，用三元组 $（I_u, P_u,F_i）$ 表示user-item 实例，\
其中 $I_u$ 为用户对Item 产生行为序列集合， $P_u$ 为用户基本属性（如性别、年龄、）， $F_i$  为目标Item 的基本属性（如itemid, category_id）.\
MIND 主要任务就是学习一个函数可以将user-item 实例映射为用户的向量表达：\
$V_u = f_user(I_u, P_u)$\
$V_u = (v_u^1, v_u^2....v_u^K)$  即为用户多兴趣的向量表达，其中K 表示用户K 个兴趣向量。\
$e_i = f_item(F_i)$ 表示item的向量\
TopN 候选Item 可以通过dot-product 计算得出：\
$f_score(V_u, e_i) = max_{1≤k≤K} e_i * v_u^k$


### Label-aware Attention Layer.

通过多兴趣提取器层，对用户行为序列embedding 我们得到多个个兴趣Capsule 表达用户多样的兴趣分布，不同的兴趣Capsule表示用户兴趣的不同偏好。为了评估多个兴趣Capsule对目标Item 相关度及贡献度，我们设计标签意识的Attention 机制来衡量目标Item 选择使用哪个兴趣Capsule：

$v_u = Attention(e_i, V_u, V_u) = V_u * softmax(pow(V_u^t * e_i, p))$


### 损失函数：

使交叉熵作为MIND 训练的损失函数：

$L = \sum_{(u,i)\in D} \log Pr(i|u)$
$ Pr(i|u) = Pr(e_i|v_u) =  \frac{exp(v_u^t * e_i)}{\sum_{j\in I} exp(v_u^t * e_j)}$


##  capsule network 胶囊网络
提取用户兴趣向量主要借鉴的方法是capsule network（胶囊网络），因此在进入Multi-Interest Extractor Layer讲解之前，需要搞清楚capsule network的原理。

capsule network由low-level的capsule和high-level的capsule组成，目的在于通过 Dynamic Routing（动态路由）的方式，根据low-level capsule来计算得到high-level的 capsule。

<img src="../data/img/capsule_vs_nn.png" style="zoom:50%"/> 

<img src="../data/img/capsule_routing.png" style="zoom:50%"/>



另外还有几个细节：

- $b_{ij}$一般是初设化为0；
- 为了能够收敛，整个动态路由过程一般重复3次，即以上由low-level capsule得到high-level capsule的过程；
- 由于high-level capsule是capsule network的输出，我们的输入只有low-level capsule，所以第一次routing $b_{ij}$无法计算得到 
 ，才需要将 $b_{ij}$ 初设化为0；
- 借由第一次routing过程计算得到的high-level capsule，就可以给到后面的routing过程中了。

CapsNet由两部分组成：编码器和解码器。前3层是编码器，后3层是解码器：

### encoder
第一层：卷积层
第二层：PrimaryCaps（主胶囊）层
第三层：DigitCaps（数字胶囊）层
第四层：第一个全连接层
第五层：第二个全连接层
第六层：第三个全连接层

<img src="../data/img/capsule_encoder.png" style="zoom:50%"/>

### loss function
<img src="../data/img/capsule_loss.png" style="zoom:50%"/>


### decoder

<img src="../data/img/capsule_decoder.png" style="zoom:50%"/>

## B2I Dynamic Routing
论文对经典的capsule network作了一些调整，称为Behavior-to-Interest (B2I) dynamic routing，可以从字面意思理解，就是通过用户的行为，通过动态路由的方法得到用户兴趣向量。

在这个场景下，low-level capsule对应用户的行为，更具体点，就是用户发生交互的item向量；

high-level capsule对应用户的多个兴趣capsule。


$
\begin{align}
\displaystyle
& low-level capsule： c_i^l \in R^{N_l x 1} , i \in \{1,2,3,....m\} \\
& high-level capsule：  c_j^h \in R^{N_h x 1} , i \in \{1,2,3,....m\} \\
& routing logit： b_{ij} = (c_j^h)^T S_{ij} c_i^l \\
& 其中 S_{ij} \in R^{N_h x N_l}  为双线性映射矩阵，为可被学习参数矩阵\\
& 可以得到high-level capsule的候选向量：\\
& z_j = \sum_{i=1}^m w_{ij} S_{ij}c_i^l \\
& 是连接high-level capsule和low-level capsule的权重：\\
& w_{ij} = softmax(b_{ij}) = \frac {exp(b_{ij})}{\sum_k^m exp(b_{ik})}\\
& 最终高阶Capsule 输出向量：\\
& c_j^h = squash(z_{j}^h) = \frac{||z_{j}^h|| ^2}{1 + ||z_{j}^h||^2} \frac{z_{j}^h}{||z_{j}^h||} \\
& Squash函数是Hinton专门为胶囊网络设计的新的激活函数。\\
\end{align}
$


调整的包括以下3个方面：

- A. Shared bilinear mapping matrix：不同于经典的capsule network中每一对low-level capsule和high-level capsule的共享双线性映射矩阵S是不同的；

这篇论文调整为共享双线性映射矩阵S，一方面是Tmall用户行为长度都在数百以内，所以认为双线性映射矩阵S是可以泛化的；另一方面是希望用户兴趣capsule能够处于相同的向量空间，但不同的双线性映射矩阵S会导致用户兴趣capsule映射到不同向量空间。

- B. Randomly initialized routing logits：前面提到routing logit $b_{ij}$ 初设化为0，但由于共享了双线性映射矩阵S，这就会导致相同的初设兴趣capsule，即第一次动态路由iteration时K个初设兴趣capsule相同。所以将 $b_{ij}$ 初设化修改为高斯分布 $N(0, \delta ^2)$

- C. Dynamic interest number：不同的用户设置不同的兴趣capsule数量，这主要是为了节省计算资源：\
   $K_u^{'} = max(1, min(K, \log_2(|I_u|)))$


**MIND的routing logit  $b_{ij}$ 在迭代的过程中是累加的，见下图第7点，但论文中原始的routing公式是没有的，即上述的这个公式：（感觉可能是论文公式漏了加上累加）**

<img src="../data/img/mind_B2I.webp" style="zoom:50%"/>